In [356]:
import pandas as pd
import numpy as np
import csv
import re

#Deberia revisar lo de Counter que lo uso bastante y no conoxco muy bien la libreria
from collections import Counter


In [261]:
def format_price(x):
    
    """
    Decompose and fromate values for the colum price
    
    input: string 
    output: lsit, values of each key already casted:
        info: pos[0]: str 
        valores: pos[1]: float
        descuento: pos[2]: float/np.nan(float) (where there is a discount or not)
    
    """
    
    # 1. Replace the € simbol and splits by the "/" ch -- generates a list of 2 elements (nums - text)
    x = x.replace("€", "").split("/")
    
    # 2. Split by the "/" to extract the info, keep last part(always at the end)
    info = x[-1].replace(".", "") #(str)
    
    # 3. Splits by the "/" to extract the num values:
    num_values = x[0:-1][0].replace("," , ".").strip().split(" ")
    
    # 4. Extracts and formats price value:
    price = float(num_values[0])
    
    # 5. Extracts and formats discount_price if not dicount np.nan
    discount = [np.nan if len(num_values) == 1 else float(num_values[1])][0]
    
    return [info, price, discount]

In [6]:
ls

scrape-consum_2024-03-10.csv    srape-mercadona_2024-03-08.csv


In [276]:
merca = pd.read_csv("srape-mercadona_2024-03-08.csv")
df = merca

In [81]:
df.sample()

#categoria, subcategoria y descripcion - las dejamos como están. 

,categoria,subcategoria,descripcion,unidades,precio
1039,Bodega,Licores,Pacharán Oibar,Botella 1 L,"6,90 € /ud."


In [82]:
# All collumns have values. 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5035 entries, 0 to 5034
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   categoria     5035 non-null   object
 1   subcategoria  5035 non-null   object
 2   descripcion   5035 non-null   object
 3   unidades      5035 non-null   object
 4   precio        5035 non-null   object
dtypes: object(5)
memory usage: 196.8+ KB


### Precio

- No hay valores nulos

- El precio puede contener o 2 informaciones o 3:
    - El precio y a que hace referencia ese precio (pack)
    - El precio, precio si esta descontado y a que hace referencia ese precio(pack)

- Creamos una funcion que aplicamos a todos los elementos de la columna precio para formaterla. Aqui extraeremos los tres valores comentados antes. Cuando no haya descuento --> np.nan
    - price_info --> string
    - price_eu and price_disc_eu --> float
    
  **Note:** Interesante aqui el tema de apply(function) y luego el lambda. 
  
- Eliminamos la columna original

**NOTE:** En price_info, los valores son bastante consistentes siendo la mayoria de valoes (ud/pack). No obstante hay 14 valores que son de tipo ( xxx g) ej 500 g. Veremos a ver que hacemos. si los descartamos o como los tranformamos. No son una parte muy relevante del conjunto de datos ( 14 de 5035) pero tener en cuenta que hay que revisarlo: 


**Tranform Price column**

In [277]:
# Apply the function and get the element for each colum: 
df["price_info"] = df["precio"].apply(format_price).apply(lambda x:x[0])
df["price_eu"] = df["precio"].apply(format_price).apply(lambda x:x[1])
df["price_disc_eu"] = df["precio"].apply(format_price).apply(lambda x:x[2])

#Drop old column:
df.drop("precio", axis=1, inplace=True)

df.sample()

### UNIDADES:

In [289]:
df["unidades"].value_counts()

unidades
1 ud.                252
Caja 1 ud.           153
Botella 750 ml       121
Brick 1 L             89
Paquete 500 g         85
                    ... 
Paquete 16 bandas      1
Caja 320 g             1
Caja 200 ml            1
Caja 150 ml            1
Bandeja 450 g          1
Name: count, Length: 1396, dtype: int64

In [288]:
lista = df["unidades"].unique()
len(lista)

1396

In [339]:
lista
re_words = ["x", "aprox." "(g", "g)", "g", "ud.", "ml", "kg", "ml", "L", "ml)", "(250"]
numb = 30
words = []

words_list = [ i.strip().split(" ") for i in lista]

for i in words_list:
    for e in i:
        if e.isdigit() == False and e not in re_words:
            words.append(e)
            
word_counts = Counter(words)
    
most_common_words = word_counts.most_common(numb)

palabras = [i[0] for i in most_common_words]

In [341]:
print(palabras)

['Paquete', 'Caja', 'aprox.', 'Pieza', 'Bote', 'Bandeja', 'Tarro', 'escurrido)', 'paquetes', 'Botella', 'Tarrina', 'cápsulas', 'latas', 'escurrido', 'sobres', 'Lata', 'lavados', 'botellas', 'raciones', 'Spray', 'bolsitas', 'Tubo', 'Bol', 'rebanadas', '(240', 'mini', '(200', 'barritas', 'Sobre', '1,5']


In [351]:
f_df = df[df['unidades'].str.contains('Paquete')]

In [349]:
Paquete = i_df["unidades"].tolist()

In [357]:
def texto_csv(df, column):
    
    #name of the document: as column_name_column.csv
    path = f'column_{column}.csv'
    
    """ Hay que quitar los vacios primero - lo he sacado de la funcion
    por no tranformar la tabla aqui """
   
    #2. We write every colum as a new line in the .csv
    non_empty_values = [str(i) for i in df[column]]
    
    with open(path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows([[value] for value in non_empty_values]) 
        
    print(f'Column successfully exported to {path}')

In [358]:
texto_csv(f_df, "unidades")

Column successfully exported to column_unidades.csv
